# Naive representation pipeline (pennylane)

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [1]:
import numpy as np
import math
import sys
import struct

import pennylane as qml
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
from pennylane.templates.embeddings import AmplitudeEmbedding

## Generating an artificial image of the sky

In [2]:
#image of the sky filled with single precision complex floats
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype=complex)
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

NameError: name 'circular_mask' is not defined

## Applying 2D Fourier transform (visibilities)

In [ ]:
visibilities = np.fft.fft2(sky_image)
print(visibilities)

### Sanity check

In [ ]:
img = np.fft.ifft2(visibilities)
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

## Classical data encoding/decoding

In [ ]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
data = visibilities.flatten()
print(data)
print()

data_reals = []
data_imags = []
for i in range(0, len(data)):
    data_reals.append(float_to_bin_real(data[i].real))
    data_imags.append(float_to_bin_imag(data[i].imag))
data_binaries = data_reals + data_imags

data_binary = []
for i in range(0, len(data_binaries)):
    data_binary.append([*data_binaries[i]])
    
inp = []
for i in range(0, len(data_binary)):
    inp += data_binary[i]

print(inp)

wires = range(len(inp)) # set the number of qbits (no padding needed if outputs an integer)
amp_dev = qml.device('default.qubit', wires, shots=1)

@qml.qnode(basis_dev)
def basis_encoding(data):
    qml.BasisEmbedding(data, wires)
    return qml.state
    
amp_encoding(inp)
readout = amp_dev.state
readout = np.array(readout).reshape(n , n)
print(readout)

## Applying 2D Inverse Fourier transform (+ fidelity test)

In [ ]:
img = np.fft.ifft2(readout)

test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

## Utils

### Quantum

### Classical

In [ ]:
#float to binary / binary to float
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask